## ComCam Donuts

Looking at ComCam donuts vs M2 Hexapod position
Craig Lage - 30-Oct-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.summit.utils.plotting import plot
from astropy.time import Time, TimeDelta
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst_efd_client import EfdClient

In [ ]:
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib", "LSSTComCam/nightlyValidation"])
client = EfdClient('usdf_efd')

In [ ]:
def findStamp(sources, index):
    # Centers a donut stamp that was found with CharacterizeImageTask
    stamp1 = 250
    x11 = int(sources[index]['base_SdssCentroid_x']) - stamp1
    x21 = int(sources[index]['base_SdssCentroid_x']) + stamp1
    y11 = int(sources[index]['base_SdssCentroid_y']) - stamp1
    y21 = int(sources[index]['base_SdssCentroid_y']) + stamp1
    first = exp.image.array[y11:y21,x11:x21]
    first -= np.nanmedian(first)    

    sumx = 0.0
    sumy = 0.0
    for i in range(first.shape[0]):
        for j in range(first.shape[1]):
            if np.isnan(first[i,j]):
                continue
            sumx += i * first[i,j]
            sumy += j * first[i,j]
    sumx /= np.nansum(first)
    sumy /= np.nansum(first)
    stamp2 = 120
    x12 = int(sumx-stamp2)
    x22 = int(sumx+stamp2)
    y12 = int(sumy-stamp2)
    y22 = int(sumy+stamp2)
    second = first[x12:x22, y12:y22]
    return second


charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = False   
charConfig.detection.minPixels = 10000
charConfig.doNormalizedCalibration=False
charTask = CharacterizeImageTask(config=charConfig)


## First the XY sweep

In [ ]:
%matplotlib inline
instrument = 'LSSTComCam'
data = []   
for expId in range(2024102600179, 2024102600198):
    try:
        exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)
        mData = exp.getMetadata()
        charResult = charTask.run(exp)
        sourceCatalog = charResult.sourceCat
        sources = sourceCatalog.asAstropy()
        sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
        sources.sort('base_CircularApertureFlux_3_0_instFlux', reverse=True)

        print(f"{sources[0]['base_SdssCentroid_x']:.1f}, {sources[0]['base_SdssCentroid_y']:.1f}") 
        stamp = findStamp(sources, 0)
    
        start = Time(mData['DATE-BEG'], scale='tai')
        end = Time(mData['DATE-END'], scale='tai')
        mhex = await client.select_time_series("lsst.sal.MTHexapod.application", 
                                     ["*"], start.utc, end.utc, index=2)
        X = np.median(mhex["position0"].values)
        Y = np.median(mhex["position1"].values)
        data.append([X, Y, stamp])
    except:
        print(f"{expId} failed")
        continue


In [ ]:
len(data)

In [ ]:
for i in range(len(data)):
    X = data[i][0]
    Y = data[i][1]
    print(f"{X:.1f}, {Y:.1f}")

In [ ]:
fig = plt.figure(figsize=(14.5,10))
big_ax = plt.axes([0.1,0.1,0.8,0.8])

xmin = -1500
xmax = 800
ymin = -750
ymax = 900
big_ax.set_title("M2 Sweep 2024-10-26", fontsize=24)
big_ax.set_xlim(xmin, xmax)
big_ax.set_ylim(ymin, ymax)
big_ax.set_xlabel("M2 Hex X position (microns)", fontsize=18)
big_ax.set_ylabel("M2 Hex Y position (microns)", fontsize=18)
size = 0.10
for i in range(len(data)):
    X = data[i][0]
    Y = data[i][1]
    xcen = 0.1 + (X-xmin) / (xmax-xmin) * 0.8
    ycen = 0.1 + (Y-ymin) / (ymax-ymin) * 0.8
    stamp = data[i][2]

    ax = plt.axes([xcen-size/2.0,ycen-size/2.0,size,size])
    ax.set_aspect(1.0)
    ax.imshow(stamp)
    ax.set_axis_off()
plt.savefig(f"/home/c/cslage/u/MTAOS/images/M2_Sweep_2024-10-26.png")

## Next, the UV sweep

In [ ]:
%matplotlib inline
instrument = 'LSSTComCam'
data = []   
for expId in range(2024102600116, 2024102600153):
    if expId == 2024102600134:
        continue
    try:
        exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument=instrument)
        mData = exp.getMetadata()
        charResult = charTask.run(exp)
        sourceCatalog = charResult.sourceCat
        sources = sourceCatalog.asAstropy()
        sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
        sources.sort('base_CircularApertureFlux_3_0_instFlux', reverse=True)
    
        print(f"{sources[0]['base_SdssCentroid_x']:.1f}, {sources[0]['base_SdssCentroid_y']:.1f}") 
        stamp = findStamp(sources, 0)
    
        start = Time(mData['DATE-BEG'], scale='tai')
        end = Time(mData['DATE-END'], scale='tai')
        mhex = await client.select_time_series("lsst.sal.MTHexapod.application", 
                                     ["*"], start.utc, end.utc, index=2)
        U = np.median(mhex["position3"].values)
        V = np.median(mhex["position4"].values)
        data.append([U, V, stamp])
    except:
        print(f"{expId} failed")
        continue


In [ ]:
len(data)

In [ ]:
for i in range(len(data)):
    U = data[i][0] * 3600
    V = data[i][1] * 3600
    print(f"{U:.2f}, {V:.2f}")

In [ ]:
fig = plt.figure(figsize=(14.5,10))
big_ax = plt.axes([0.1,0.1,0.8,0.8])

xmin = -250
xmax = 250
ymin = -250
ymax = 300
big_ax.set_title("M2 Tilt Sweep 2024-10-26", fontsize=24)
big_ax.set_xlim(xmin, xmax)
big_ax.set_ylim(ymin, ymax)
big_ax.set_xlabel("M2 Hex U position (arcsec)", fontsize=18)
big_ax.set_ylabel("M2 Hex V position (arcsec)", fontsize=18)
size = 0.10
for i in range(len(data)):
    if i % 2 == 0:
        continue
    U = data[i][0] * 3600
    V = data[i][1] * 3600
    xcen = 0.1 + (U-xmin) / (xmax-xmin) * 0.8
    ycen = 0.1 + (V-ymin) / (ymax-ymin) * 0.8
    stamp = data[i][2]

    ax = plt.axes([xcen-size/2.0,ycen-size/2.0,size,size])
    ax.set_aspect(1.0)
    ax.imshow(stamp)
    ax.set_axis_off()
plt.savefig(f"/home/c/cslage/u/MTAOS/images/M2_Tilt_Sweep_2024-10-26.png")